# **Real Estate Dataset** #

## **Project Overview** ##

The Office of Policy and Management maintains a listing of all real estate sales with a sales price of $2,000 or greater that occur between October 1 and September 30 of each year. For each sale record, the file includes: town, property address, date of sale, property type (residential, apartment, commercial, industrial or vacant land), sales price, and property assessment.

## **Project Goal** ##

The main goals of our design focus on:
- Visualizing sales trends by year.
- Identifying the top 10 towns with the highest total sale amounts.
- Breaking down sales by property type each year.
- Mapping town-wise total sales to show spatial distribution.

## **Data** ##

Data are collected in accordance with Connecticut General Statutes, section 10-261a and 10-261b: https://www.cga.ct.gov/current/pub/chap_172.htm#sec_10-261a and https://www.cga.ct.gov/current/pub/chap_172.htm#sec_10-261b. Annual real estate sales are reported by grand list year (October 1 through September 30 each year). 

Some municipalities may not report data for certain years because when a municipality implements a revaluation, they are not required to submit sales data for the twelve months following implementation.

**Loading the Data**

In [1]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import the required libraries
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from dash import dcc, html, Dash, callback, Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# Loading the data
data = pd.read_csv('RealEstateSales.csv')

In [4]:
# Displaying the summary of dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097628 entries, 0 to 1097627
Data columns (total 17 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Serial Number     1097628 non-null  int64  
 1   List Year         1097628 non-null  int64  
 2   Date Recorded     1097626 non-null  object 
 3   Town              1097628 non-null  object 
 4   Address           1097577 non-null  object 
 5   Assessed Value    1097628 non-null  float64
 6   Sale Amount       1097628 non-null  float64
 7   Sales Ratio       1097628 non-null  float64
 8   Property Type     715182 non-null   object 
 9   Residential Type  699239 non-null   object 
 10  Non Use Code      313450 non-null   object 
 11  Assessor Remarks  171228 non-null   object 
 12  OPM remarks       13031 non-null    object 
 13  Location          298111 non-null   object 
 14  town_new          1097628 non-null  object 
 15  latitude          1097628 non-null  float64
 16  

This dataset consists of 1.1M rows and 17 columns. The data contains both categorical and numerical features and each row is a real estate sale. Here's a breakdown of the data:

Size:
1097628 rows (instances)
17 columns (attributes): 7 numerical, 10 categorical.

- Serial Number: Serial Number.
- List Year: Year the property was listed for sale.
- Date Recorded: Date the sale was recorded locally.
- Town: Town name.
- Address: Address.
- Assessed Value: Value of the property used for local tax assesment.
- Sale Amount: Amount the property was sold for.
- Sales Ratio: Ratio of the sale price to the assessed value.
- Property Type: Type of property including: Residential, Commercial, Industrial, Apartments, Vacant, etc.
- Residential Type: Indicates whether property is single or multifamily residential.
- Non Use Code: Non usable sale code typically means the sale price is not reliable for use in the determination of a property value. See attachments in the dataset description page for a listing of codes.
- Assessor Remarks: Remarks from the assessor.
- OPM remarks: Remarks from OPM.
- Location: Lat / lon coordinates.
- town_new: town name and the state name.
- latitude: latitude.
- longitude: longitude.

## **Data Cleaning** ##

In [5]:
# Data Cleaning
data_cleaned = data.drop(data[data['Town'] == '***Unknown***'].index)
data_cleaned['Town'].unique()

array(['Ansonia', 'Avon', 'Barkhamsted', 'Beacon Falls', 'Berlin',
       'Bethel', 'Bethlehem', 'Bolton', 'Branford', 'Bristol',
       'Brookfield', 'Burlington', 'Canton', 'Cheshire', 'Colchester',
       'Coventry', 'Cromwell', 'Danbury', 'Darien', 'Derby',
       'East Granby', 'East Haddam', 'East Hartford', 'East Haven',
       'East Lyme', 'Easton', 'East Windsor', 'Enfield', 'Essex',
       'Farmington', 'Franklin', 'Glastonbury', 'Goshen', 'Granby',
       'Greenwich', 'Griswold', 'Groton', 'Guilford', 'Hartford',
       'Harwinton', 'Killingly', 'Killingworth', 'Lebanon', 'Lisbon',
       'Litchfield', 'Madison', 'Manchester', 'Mansfield', 'Meriden',
       'Middlebury', 'Middletown', 'Milford', 'Monroe', 'Montville',
       'Naugatuck', 'New Britain', 'New Canaan', 'New Fairfield',
       'New Haven', 'New London', 'Newtown', 'North Canaan',
       'North Haven', 'Norwalk', 'Norwich', 'Old Lyme', 'Old Saybrook',
       'Orange', 'Plainfield', 'Plainville', 'Plymouth', 'Port

In [6]:
# imports external stylesheet and creates a Dash web app
css = ["https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/css/bootstrap.min.css", ]
app = Dash(name="RealEstateSales Dashboard", external_stylesheets=css)

In [7]:
# Create a table to show the sample data
def create_table():
    top_40_data = data_cleaned[['Serial Number','List Year','Date Recorded','Town', 'Address', 'Assessed Value','Sale Amount',
                                'Sales Ratio','Property Type','Residential Type','Non Use Code','Assessor Remarks','OPM remarks',
                                'Location','town_new','latitude','longitude']].head(40)

    fig = go.Figure(data=[go.Table(
        header=dict(values=top_40_data.columns, align='left', fill_color='lightgrey', font=dict(size=12)),
        cells=dict(values=top_40_data.values.T, align='left', fill_color='white', font=dict(size=11))
    )])

    # Adjust layout for better auto-fitting
    fig.update_layout(
        paper_bgcolor="#e5ecf6",
        margin={"t": 0, "l": 0, "r": 0, "b": 0},
        height=800,
        autosize=True
    )
    return fig

In [8]:
# Town Sales by Year Chart
def create_townsalesbyyear_chart(town="Andover"):
    filtered_df = data_cleaned[data_cleaned["Town"] == town]
    filtered_df = filtered_df.groupby(['Town','List Year'])['Serial Number'].count().reset_index(name = 'Number of Sales')
    
    fig = px.line(filtered_df, x="List Year", y="Number of Sales", #color="Town",
                  title=f"Sale Amount for {town} Over the Years")
    
    fig.update_layout(paper_bgcolor="#e5ecf6", height=600)
    return fig

In [9]:
# Top 5 towns by Sale Amount
def create_top5towns_chart(year=2022):
    filtered_df = data_cleaned[data_cleaned["List Year"] == year]
    filtered_df = filtered_df.groupby(['Town','List Year'])['Sale Amount'].sum().reset_index(name = 'Total Sale Amount')
    filtered_df['Total Sale Amount (Millions)'] = filtered_df['Total Sale Amount'].div(1_000_000).round(2)
    filtered_df = filtered_df.sort_values(by = 'Total Sale Amount (Millions)', ascending=False).head(10)
 
    fig = px.bar(filtered_df, x="Town", y="Total Sale Amount (Millions)", color="Town",
                   title="Top 10 Towns by Sale Amount in the year {}".format(year),
                   text_auto=True)
    fig.update_layout(paper_bgcolor="#e5ecf6", height=600)
    return fig

In [10]:
# Number of Sales by Property Type 
def create_propertysales_chart(year=2022):
    filtered_df = data_cleaned[data_cleaned["List Year"] == year]
    filtered_df = filtered_df.groupby(['Property Type','List Year'])['Serial Number'].count().reset_index(name = 'Number of Sales')
 
    fig = px.pie(filtered_df, values='Number of Sales', names='Property Type', 
                 title ='Number of Sales by Property Type in the year {}'.format(year))
    fig.update_layout(paper_bgcolor="#e5ecf6", height=600)
    
    return fig

In [11]:
filtered_df = data_cleaned.groupby(['town_new','latitude','longitude'])['Sale Amount'].sum().reset_index(name='Total Sales')
filtered_df['Total Sales (M)'] = filtered_df['Total Sales'].div(1_000_000).round(2)
# Function to create the map
def create_salesonmap_chart():
    fig = px.scatter_mapbox(
        filtered_df,
        lat="latitude",
        lon="longitude",
        size="Total Sales (M)",
        color="Total Sales (M)",
        center={"lat": 41.6032, "lon": -73.0877},
        hover_name="town_new",
        title="Town-wise Sales",
        mapbox_style="carto-positron"
        ,zoom=8
    )
    return fig

In [12]:
# Dropdown for selecting town
towns = sorted(data_cleaned["Town"].unique()) 
years = sorted(data_cleaned['List Year'].unique(),reverse = True)
listyears_old = data_cleaned.drop(data_cleaned[data_cleaned['List Year'].isin([2001, 2002, 2003, 2004, 2005])].index)
listyears = sorted(listyears_old['List Year'].unique(),reverse = True)
                   
cont_town = dcc.Dropdown(id="cont_town",options=[{"label": town, "value": town} for town in towns], value="Andover",clearable=False)
cont_year = dcc.Dropdown(id="cont_year",options=[{"label": year, "value": year} for year in years], value=2022,clearable=False)
cont_proptypeyears = dcc.Dropdown(id="cont_proptypeyears",options=[{"label": year, "value": year} for year in listyears], value=2022,clearable=False)

In [13]:
#app layout
app.layout = html.Div([
    html.Div([
        html.H1("Real Estate Sales Dataset Analysis", className="text-center fw-bold m-2"),
        html.Br(),
        dcc.Tabs([
            dcc.Tab([html.Br(),
                     dcc.Graph(id="dataset", figure=create_table())], label="Dataset"),
            dcc.Tab([html.Br(), "Select Town", cont_town, html.Br(),
                     dcc.Graph(id="townwisesales")],label="Year Wise Sales"),
            dcc.Tab([html.Br(), "Select Year", cont_year, html.Br(),
                     dcc.Graph(id="top10towns")], label="Town Wise Sales"),
            dcc.Tab([html.Br(), "Select Year", cont_proptypeyears, html.Br(),
                     dcc.Graph(id="propertytypesales")], label="Property Type Sales"),
            dcc.Tab([html.Br(), 
                     dcc.Graph(id="sales_map",figure=create_salesonmap_chart())], label="Town Wise Sales Map")
        ])
    ], className="col-8 mx-auto"),
], style={"background-color": "#3e6eb2", "height": "150vh"})

In [14]:
# Callbacks
@callback(Output("townwisesales", "figure"), [Input("cont_town", "value")])
def update_townsalesbyyear_chart(town):
    return create_townsalesbyyear_chart(town)

@callback(Output("top10towns", "figure"), [Input("cont_year", "value")])
def update_top5towns_chart(year):
    return create_top5towns_chart(year)

@callback(Output("propertytypesales", "figure"), [Input("cont_proptypeyears", "value")])
def update_propertysales_chart(year):
    return create_propertysales_chart(year)

In [15]:
if __name__ == "__main__":
    app.run(debug=True)

## **Design Goals and Justification** ##

**Visualizing Sales Trends by Year**:

Design Approach:
- Used a line chart to show how total sales fluctuate over time.
- Added interactive filters for dynamic exploration.

Justification:
- Helps identify market trends, seasonal variations, and growth patterns.
- Useful for realtors, investors, and policymakers to track sales performance.

**Identifying the Top 10 Towns with the Highest Total Sale Amounts**:

Design Approach:
- Used a bar chart ranking towns by total sales amount.
- Highlighted top towns with distinct colors.

Justification:
- Helps pinpoint high-value markets.
- Investors and businesses can focus on lucrative locations.

**Breaking Down Sales by Property Type Each Year**:

Design Approach:
- Used a pie chart to visualize the proportion of total sales by property type each year.
- Implemented dropdown filters to select different years for comparison.

Justification:
- A pie chart clearly shows the distribution of sales across property types, making it easy to see the market share of each category.
- It provides a simple, visual comparison of property type demand, aiding decision-making for developers and investors.

**Mapping Town-wise Total Sales to Show Spatial Distribution**:

Design Approach:
- Used an interactive map using Plotly.
- Colored towns based on total sales, with hover tooltips for extra details.

Justification:
- Helps visualize geographic sales disparities.
- Useful for city planners and real estate agents to identify high-performing areas.

## **Final Evaluation Approach** ##

The final evaluation involved user testing with 3 participants (friends, classmates, colleagues) who tested the dashboard's usability, clarity, and data communication effectiveness. Feedback indicated that the dashboard was intuitive and visually appealing, with positive responses to the interactive features and pie chart for sales breakdowns. Suggestions for improvement included adding detailed tooltips, annotations for trends, and optimizing color contrast for accessibility.

## **Synthesis of Findings** ##

The interactive features, such as dropdown filters and the pie chart for property sales, worked well and engaged users effectively. However, future iterations should refine tooltips and annotations for better clarity, improve color contrast for accessibility, and adjust font sizes for better readability.